# Multiple Linear Regression Code Example

The goal with the walk through is to introduce you to the [scikit-learn](https://scikit-learn.org/stable/) environment for *predictive* modeling.

At the end of this, you should be comfortable with basics of scikit-learn model instantiation, training, and predicting.

You have been asked by HR to help them come up with a model for salaries for new position your company is hiring for. Let's follow the steps of the [CRISP-DM](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining) process.

--insert graphic here--
    

### Business Understanding

Fortunately, this is pretty straight forward: how much to pay people?

### Data Understanding

We are going to take a short-cut, and use a pre-prepared data set from the most recent [Stack OverFlow Developers Survey](https://survey.stackoverflow.co/2024/) from 2024. Here is, however the data dictionary:

| **Column**  | **Data Type** | **Description**                        |
|-------------|---------------|----------------------------------------|
| Age         | object        | The age binned into several categories |
| RemoteWork  | object        | What kind of remote work               |
| EdLevel     | object        | Education level                        |
| YearsCode   | int64         | How many years coding                  |
| primaryDB   | object        | Primary Database Used                  |
| primaryLang | object        | Primary language used                  |
| Salary      | float64       | Salary in USD                          |

Looking at this, it is obvious that we are going to need to create dummy variables for most of these columns.

### Data Preparation

Again, this has been done for you, in order to facilitate this exercise. The on additional step we will use is [`pd.get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) in order to turn the object columns into numeric variables.

### Modeling

We will be using [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) for our model

### Evaluation

We will be looking at two metrics to determine how effective our model is for predictive purposes.

1. [$r^2$ score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)
2. [mean squared error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)



### Deployment

For our purposes, deployment will be completing this notebook, and successfully predicting a salary.

# Data import and examination and preparation

Ready? Excellent, let's get to the code!

In [1]:
# imports
# best practice is to keep all the imports in once cell, at the top of your notebook or script

import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [12]:
# Bring the prepared data in

df = pd.read_csv('cleanedSO.csv')

 Using some of the basic pandas tools, explore this dataset. Think back to descriptive statistics and meta data about the dataframe.

In [13]:
df.shape

(21807, 7)

In [14]:
df.describe()

,YearsCode,Salary
count,21807.000000,21807.000000
mean,15.685055,90012.179599
std,9.926717,68987.446492
min,1.000000,0.024000
25%,8.000000,42560.000000
50%,13.000000,75000.000000
75%,20.000000,120000.000000
max,50.000000,498000.000000


In [15]:
df.describe(include='object')

,Age,RemoteWork,EdLevel,primaryDB,primaryLang
count,21807,21807,21807,21807,21807
unique,8,3,8,8,11
top,25-34 years old,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",other,Bash/Shell (all shells)
freq,9233,10001,9646,12370,7735


In [16]:
df.dtypes

Age             object
RemoteWork      object
EdLevel         object
YearsCode        int64
Salary         float64
primaryDB       object
primaryLang     object
dtype: object

In [17]:
df.isna().sum()

Age            0
RemoteWork     0
EdLevel        0
YearsCode      0
Salary         0
primaryDB      0
primaryLang    0
dtype: int64

#### Questions to consider:

1. Does anything jump out while looking at the descriptive statistics?
2. What columns can we use without manipulation?
3. How are we going to handle the non-numerics?

### Data Preparation

Remember those non-numeric variables? We need to deal with them. We will do that by creating [dummy variables](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)) using the built-in pandas method [`pd.get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

In [20]:
# pd.get_dummies is smart enough to look at the datatypes and only use the fields 
# that are object fields.
df = pd.get_dummies(df)

In [22]:
df.head()

,YearsCode,Salary,Age_18-24 years old,Age_25-34 years old,Age_35-44 years old,Age_45-54 years old,Age_55-64 years old,Age_65 years or older,Age_Prefer not to say,Age_Under 18 years old,...,primaryLang_Bash/Shell (all shells),primaryLang_C,primaryLang_C#,primaryLang_C++,primaryLang_Go,primaryLang_HTML/CSS,primaryLang_Java,primaryLang_JavaScript,primaryLang_Python,primaryLang_other
0,12,31360.0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,15,95200.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,27,56000.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,7,110000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,32,166874.4,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
# check the datatypes!
df.dtypes

YearsCode                                                                                       int64
Salary                                                                                        float64
Age_18-24 years old                                                                             uint8
Age_25-34 years old                                                                             uint8
Age_35-44 years old                                                                             uint8
Age_45-54 years old                                                                             uint8
Age_55-64 years old                                                                             uint8
Age_65 years or older                                                                           uint8
Age_Prefer not to say                                                                           uint8
Age_Under 18 years old                                                            

Note that after "dummying" we now have all numeric columns

# Modeling

We will now instantiate, train, and use a model to make predictions.

scikit-learn as a large number of methods, which we will not exploring in their entirety here. We will focus on two methods:
1. [`.fit()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit)
2. [`.score()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score)

`.fit()` is the method we will use to train our linear regression algorithm.

`.score()` will return the default scoring method, which, for linear regression is $r^2$.

In [24]:
# instantiate
# this creates a linear regression object that we can train and score

lr = LinearRegression()

Statsmodels uses terms like endogenous and exogenous variables. scikit-learn uses different terminology, more in keeping with the predictive nature.

The dependent variable, or, what we are trying to predict, is always `y`.

The independent variables, or, what we are using to predict `y` is called `X`.

Note the `y` is lower case and the `X` is upper case. This is in following with matrix naming conventions. `X` is a matrix of independent variables and `y` is a vector for the dependent variable.

In [25]:
# Set the target, or the y variable

y = df['Salary']

In [27]:
# Set the independent variable matrix, X

X = df.drop('Salary', axis=1)

We now need to train the model. This is the process of applying the data to linear regression linear equation. Recall the closed form solution for linear regression.

$\vec{b} = (X^TX)^{-1}X^Ty$

Where $\vec{b}$ are the coefficients we are training. This is done via the `.fit()` method.

In [28]:
# Train the model with the .fit() method

lr.fit(X,y)

LinearRegression()

# Evaluation

We are going to use the scikit-learn metrics $r^2$ and mean squared error to evaluate how the model is performing.

In [29]:
# use .score(). Recall this returns the default scoring method, which, for linear regression is r^2.

lr.score(X,y)

0.22852359851221993

Looking at that number, what are your take-aways?





Now we'll look at mean squared error. In order to get this metric, we need to create our predictions using our trained model. This is done with the [`.predict()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) method.

In [30]:
y_pred = lr.predict(X)

In [32]:
# Use y_pred and y with the mean squared error method
mse = mean_squared_error(y, y_pred)
mse

3671494404.763504

That number is ridiculously large. The reason is this method returns the *squared* error. This doesn't really translate well, so we'll take the square of it to get a more usable number.

In [37]:
rmse = mse**(1/2)
rmse

60592.857704217124

In [38]:
# You can also get this number by setting a flag when calling the function
rmse2 = mean_squared_error(y, y_pred, squared=False)
rmse2

60592.857704217124

Our $r^2$ is .2285. This is a very low number.

Our root mean squared error is 60,592. This can be read as the average salary prediction is off by $60k. This is not good, either.

So what can we conclude from this?

1. The model isn't good. At all.
2. There is an issue with the data (a minimum salary of .024?)

# Conclusion

You now know how to:
1. instantiate a scikit-learn model
2. train a model
3. score a model with multiple metrics